<a href="https://colab.research.google.com/github/AllanKamimura/Pai_ta_ON_python/blob/main/2021_SME0340_ODE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title ## Condições Analisadas { run: "auto", form-width: "20%", display-mode: "form" }
#@markdown $A \rightleftharpoons {B} \rightarrow C$

#@markdown ---
#@markdown ### Selecione a Aproximação
#@markdown **Estado Quase Estacionario:** k2 >> km1 > k1 \
#@markdown **Aproximacao Pre-Equilibrio:** k1 > km1 >> k2 \
#@markdown **k_-1 proximo de 0:** k1 = k2 > km1 \

options = "k_-1 proximo de 0" #@param ["Estado Quase Estacionario", "Aproximacao Pre-Equilibrio", "k_-1 proximo de 0"]
#@markdown ---

if options == "Estado Quase Estacionario":
    A0 = 1
    B0 = 0
    C0 = 0

    k1 = 1
    km1 = 10
    k2 = 1000

if options == "Aproximacao Pre-Equilibrio":
    A0 = 1
    B0 = 0
    C0 = 0

    k1 = 100
    km1 = 10
    k2 = 1

if options == "k_-1 proximo de 0":
    A0 = 1
    B0 = 0
    C0 = 0

    k1 = 1
    km1 = 0.1
    k2 = 1

import sympy

t = sympy.symbols("t")

lamb1 = sympy.symbols("lambda1")
lamb2 = sympy.symbols("lambda2")
lamb3 = sympy.symbols("lambda3")

A = sympy.Function("A")
B = sympy.Function("B")
C = sympy.Function("C")

X = sympy.Matrix([A(t),B(t),C(t)])
dX = sympy.diff(X, t)

M = sympy.Matrix([[-k1, km1, 0], [k1, -k2-km1, 0], [0, k2, 0]])

xh = sympy.solvers.ode.dsolve(dX - M*X, ics = {A(0) : A0, B(0) : B0, C(0) : C0})

import numpy as np

def matrix(t_eval):
    return np.array([sympy.lambdify(t, xh[0].rhs, "numpy")(t_eval),
                     sympy.lambdify(t, xh[1].rhs, "numpy")(t_eval),
                     sympy.lambdify(t, xh[2].rhs, "numpy")(t_eval)])

import pandas as pd

df = pd.DataFrame(
    map(matrix, np.linspace(0,5,50)),
    columns = ["A(t)", "B(t)", "C(t)"]
    )

import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(y = df["A(t)"],
                    mode = "lines+markers", 
                    name = "A(t)"))

fig.add_trace(go.Scatter(y = df["B(t)"],
                    mode = "lines+markers", 
                    name = "B(t)"))

fig.add_trace(go.Scatter(y = df["C(t)"],
                    mode = "lines+markers", 
                    name = "C(t)"))
fig.show()

sympy.Matrix(xh)

Matrix([
[                  Eq(A(t), 0.421913119055698*exp(-1.37015621187164*t) + 0.578086880944303*exp(-0.729843788128358*t))],
[                   Eq(B(t), -1.56173761888606*exp(-1.37015621187164*t) + 1.56173761888606*exp(-0.729843788128358*t))],
[Eq(C(t), 0.999999999999998 + 1.13982449983037*exp(-1.37015621187164*t) - 2.13982449983036*exp(-0.729843788128358*t))]])

In [3]:
#@title ## Parametros da Reação e Condições iniciais { run: "auto", form-width: "20%", display-mode: "form" }
#@markdown $A \rightleftharpoons {B} \rightarrow C$

#@markdown ---
#@markdown ### Condição Inicial
A_0 = 1 #@param {type:"slider", min:0.05, max:1, step:0.05}
B_0 = 0.6 #@param {type:"slider", min:0.05, max:1, step:0.05}
C_0 = 0.05 #@param {type:"slider", min:0.05, max:1, step:0.05}
#@markdown ---
#@markdown ### Constantes de Equilibrio
k1 = 1 #@param {type:"number"}
km1 = 10.0 #@param {type:"number"}
k2 = 1000 #@param {type:"number"}

A0 = A_0 / (A_0 + B_0 + C_0)
B0 = B_0 / (A_0 + B_0 + C_0)
C0 = C_0 / (A_0 + B_0 + C_0)

import sympy

t = sympy.symbols("t")

lamb1 = sympy.symbols("lambda1")
lamb2 = sympy.symbols("lambda2")
lamb3 = sympy.symbols("lambda3")

A = sympy.Function("A")
B = sympy.Function("B")
C = sympy.Function("C")

X = sympy.Matrix([A(t),B(t),C(t)])
dX = sympy.diff(X, t)

xh = sympy.solvers.ode.dsolve(dX - M*X, ics = {A(0) : A0, B(0) : B0, C(0) : C0})

import numpy as np

def matrix(t_eval):
    return np.array([sympy.lambdify(t, xh[0].rhs, "numpy")(t_eval),
                     sympy.lambdify(t, xh[1].rhs, "numpy")(t_eval),
                     sympy.lambdify(t, xh[2].rhs, "numpy")(t_eval)])

import pandas as pd

df = pd.DataFrame(
    map(matrix, np.linspace(0,5,50)),
    columns = ["A(t)", "B(t)", "C(t)"]
    )

import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(y = df["A(t)"],
                    mode = "lines+markers", 
                    name = "A(t)"))

fig.add_trace(go.Scatter(y = df["B(t)"],
                    mode = "lines+markers", 
                    name = "B(t)"))

fig.add_trace(go.Scatter(y = df["C(t)"],
                    mode = "lines+markers", 
                    name = "C(t)"))
fig.show()

sympy.Matrix(xh)

Matrix([
[                 Eq(A(t), 0.198914461771233*exp(-1.37015621187164*t) + 0.407146144289374*exp(-0.729843788128358*t))],
[                 Eq(B(t), -0.736294236557262*exp(-1.37015621187164*t) + 1.09993060019362*exp(-0.729843788128358*t))],
[Eq(C(t), 0.999999999999999 + 0.537379774786029*exp(-1.37015621187164*t) - 1.507076744483*exp(-0.729843788128358*t))]])